In [176]:
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row, widgetbox
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, LabelSet, CustomJS, Slider, Range1d, FactorRange
from bokeh.models.widgets import Select, Panel, Tabs
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import math
import copy
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from random import randint
from bokeh.sampledata.olympics2014 import data
from bokeh.charts.utils import df_from_json
from bokeh.charts import Bar
from bokeh.charts.operations import blend
from bokeh.charts.attributes import cat, color

In [19]:
output_notebook()

Loading BokehJS ...

### Data Preprocessing
1. To reduce the number of features and make them compact, I have converted all columns with a frequency and quantity of some item into a single column of volume, with a value of frequency * quantity.
2. The 'Yes' and 'No's and 'Innie' and 'Outie' have been label encoded to 1's and 0's, to make the data numerical. This label encoding has been done to the first 26 columns.

In [4]:
df = pd.read_csv('nutrition_raw_anonymized_data.csv')
df.drop('ID', axis=1, inplace=True)
le = preprocessing.LabelEncoder()
colsToEncode = range(26)
for i in colsToEncode:
    df.iloc[:,i] = le.fit_transform(df.iloc[:,i])

def prediction_model(model, data, predictors, outcome):
#     print('Predicting: ', outcome)
    #Fit the model:
    model.fit(data[predictors],data[outcome])
  
    #Make predictions on training set:
    predictions = model.predict(data[predictors])

    #Print accuracy
    accuracy = metrics.accuracy_score(predictions,data[outcome])
#     print ("Accuracy : %s" % "{0:.3%}".format(accuracy))

    #Perform k-fold cross-validation with 5 folds
    kf = KFold(n_splits=5)
    error = []
    for train, test in kf.split(data):
        # Filter training data
        train_predictors = (data[predictors].iloc[train,:])

        # The target we're using to train the algorithm.
        train_target = data[outcome].iloc[train]

        # Training the algorithm using the predictors and target.
        model.fit(train_predictors, train_target)

        #Record error from each cross-validation run
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))

    #Fit the model again so that it can be refered outside the function:
    model.fit(data[predictors],data[outcome])
    return np.mean(error)

predictor_var = list(df.columns.values)

for i in range(26,281):
    if 'FREQ' in predictor_var[i]:
        df.iloc[:,i] = df.iloc[:,i]*df.iloc[:,i+1]
        predictor_var[i] = predictor_var[i].replace('FREQ', 'VOL')

df.columns = predictor_var
for i in range(len(predictor_var)):
    if 'QUAN' in predictor_var[i]:
        df.drop(predictor_var[i], axis=1, inplace=True)

## Analysis
1. 3 outcomes have been taken, namely cancer, heart disease, and diabetes. These outcomes have been predicted using a variable number of factors for each (determined throught multiple runs of the program).
2. These predictor variables are selected randomly and evaluated, and this is run multiple times and the maximum 5-fold accuracy is saved across the runs. This is done in the hope that a random selection will result in good predictor variables if run multiple times, since 1000 variables cannot be looked at manually.
3. The columns used for predicting are also kept, either the first 26 (yes and no columns) or the 164 odd columns that represent dietary habits.
4. A random forest is used and it parameters initialised with non-default values if insufficient results are obtained. Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random decision forests correct for decision trees' habit of overfitting to their training set.

In [195]:
outcome_var = 'cancer'
model = RandomForestClassifier()
predictor_var = list(df.columns.values)
# predictor_var = ['dog', 'favCable', 'cat', 'quit_smoking', 'readingMath'] #0-25
# predictor_var = ['EATMEAT']
max_CV = 0
max_predictors = []
for i in range(100):
    random_predictor = []
    for j in range(4):
        random_predictor.append(predictor_var[randint(1,26)])
    cv = prediction_model(model, df, random_predictor, outcome_var)
    if cv > max_CV:
        max_CV = cv
        max_predictors = random_predictor
        
print (max_CV, max_predictors)

0.76 ['heart_disease', 'favCable', 'readingMath', 'smoke_rarely']


In [17]:
outcome_var = 'cancer'
model = RandomForestClassifier(min_samples_split=6, max_depth=5, max_features=1)
predictor_var = ['OTHERBREADSVOL', 'WHOLEGRAINCRACKERSVOL', 'smoke_often', 'favCable']
cv = prediction_model(model, df, predictor_var, outcome_var)
featimp = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print(cv)
print(featimp)
predictor_var = ['OTHERBREADSVOL', 'WHOLEGRAINCRACKERSVOL']
predictor_var = ['heart_disease', 'favCable']

0.834545454545
WHOLEGRAINCRACKERSVOL    0.440442
OTHERBREADSVOL           0.299298
favCable                 0.211849
smoke_often              0.048411
dtype: float64


In [42]:
cancer_column = df['cancer']
cancer_colors = []
for i in range(len(cancer_column)):
    if cancer_column[i] == 1:
        cancer_colors.append('red')
    else:
        cancer_colors.append('green')
        
# cancer_source1 = ColumnDataSource(data=dict(
#     x = df['OTHERBREADSVOL'], y = df['WHOLEGRAINCRACKERSVOL'], colors = cancer_colors
# ))
y_wholegrain = [0,0]
for i in range(54):
    if df['cancer'][i] == 1 and df['WHOLEGRAINCRACKERSVOL'][i] < 3:
        y_wholegrain[0]+=1
    if df['cancer'][i] == 1 and df['WHOLEGRAINCRACKERSVOL'][i] > 3:
        y_wholegrain[1]+=1
y_bread = [0,0]
for i in range(54):
    if df['cancer'][i] == 0 and df['OTHERBREADSVOL'][i] < 5:
        y_bread[0]+=1
    if df['cancer'][i] == 0 and df['OTHERBREADSVOL'][i] > 5:
        y_bread[1]+=1
p1 = figure(plot_width = 400, plot_height = 300, x_range=['Vol < 3', 'Vol > 3'])
p1.vbar(x=[1,2], top=y_wholegrain, width=.2, color=['red','green'])
p2 = figure(plot_width = 300, plot_height = 300, x_range=['Vol < 5', 'Vol > 5'])
p2.vbar(x=[1,2], top=y_bread, width=.2, color=['red','green'])
p1.xaxis.axis_label = 'Whole Grain Crackers Vol < 3 and > 3'
p2.xaxis.axis_label = 'Other Breads Vol < 5 and > 5'
p1.yaxis.axis_label = 'Patients having Cancer'
p2.yaxis.axis_label = 'Patients having Cancer'

factors = ['Whole Grain Crackers Vol', 'Other Breads Vol', 'favCable', 'Smoke Often']
x =  [44, 29, 21, 5]

dot = figure(title="%age Contribution of Variables to predicting Cancer (Accuracy 83%)", tools="", toolbar_location=None,
            y_range=factors, x_range=[0,100], height=300)

dot.segment(0, factors, x, factors, line_width=2, line_color="green", )
dot.circle(x, factors, size=15, fill_color="orange", line_color="green", line_width=3, )

show(column(row(p1,p2), dot))

### Cancer
1. Out of the 4 factors found to affect cancer, other breads and whole grain crackers are plotted.
2. Having more bread or whole grain crackers seems to have an increased risk of cancer.

In [82]:
outcome_var = 'diabetes'
model = RandomForestClassifier()
predictor_var = list(df.columns.values)
# predictor_var = ['dog', 'favCable', 'cat', 'quit_smoking', 'readingMath'] #0-25
# predictor_var = ['EATMEAT']
max_CV = 0
max_predictors = []
for i in range(500):
    random_predictor = []
    for j in range(2):
        random_predictor.append(predictor_var[randint(3,170)])
    cv = prediction_model(model, df, random_predictor, outcome_var)
    if cv > max_CV:
        max_CV = cv
        max_predictors = random_predictor
        
print (max_CV, max_predictors)

0.834545454545 ['OYSTERSVOL', 'PEANUTBUTTERVOL']


In [98]:
model = RandomForestClassifier()
cv = prediction_model(model, df, max_predictors, outcome_var)
featimp = pd.Series(model.feature_importances_, index=max_predictors).sort_values(ascending=False)
print(cv)
print(featimp)

0.796363636364
PEANUTBUTTERVOL    0.820361
OYSTERSVOL         0.179639
dtype: float64


In [88]:
diabetes_column = df['diabetes']
# cancer_colors = []
# for i in range(len(cancer_column)):
#     if cancer_column[i] == 1:
#         cancer_colors.append('red')
#     else:
#         cancer_colors.append('green')
        
# cancer_source1 = ColumnDataSource(data=dict(
#     x = df['OTHERBREADSVOL'], y = df['WHOLEGRAINCRACKERSVOL'], colors = cancer_colors
# ))
lessThanCount = 0
moreThanCount = 0
y_pb = [0,0]
for i in range(54):
    if df['diabetes'][i] == 1 and df['PEANUTBUTTERVOL'][i] < 15:
        y_pb[0]+=1
        lessThanCount+=1
    if df['diabetes'][i] == 0 and df['PEANUTBUTTERVOL'][i] < 15:
        lessThanCount+=1
    if df['diabetes'][i] == 1 and df['PEANUTBUTTERVOL'][i] > 15:
        y_pb[1]+=1
        moreThanCount+=1
    if df['diabetes'][i] == 0 and df['PEANUTBUTTERVOL'][i] > 15:
        moreThanCount+=1
        
y_pb = [100*y_pb[0]/lessThanCount,100*y_pb[1]/moreThanCount]

y_oysters = [0,0]
for i in range(54):
    if df['diabetes'][i] == 0 and df['OYSTERSVOL'][i] < 4:
        y_oysters[0]+=1
    if df['diabetes'][i] == 0 and df['OYSTERSVOL'][i] > 4:
        y_oysters[1]+=1
        
pb_source = ColumnDataSource(data=dict(
    x=[14,16], top = y_pb, legend=['Vol < Threshold', 'Vol > Threshold']
))
sources_diabetes = ColumnDataSource(data=dict(
    pb = df['PEANUTBUTTERVOL'], diabetes= df['diabetes']
))
p1 = figure(plot_width = 300, plot_height = 300)
p1.vbar(x='x', top='top', source=pb_source, width=1.5, color=['red','green'], legend='legend')
p2 = figure(plot_width = 300, plot_height = 300, x_range=['Vol < 4', 'Vol > 4'])
p2.vbar(x=[1,2], top=y_oysters, width=0.3, color=['red','green'])
p1.xaxis.axis_label = 'Peanut Butter Vol < and > threshold'
p2.xaxis.axis_label = 'Oysters effect on diabetes occurence'
p1.yaxis.axis_label = '% Patients having Diabetes'
p2.yaxis.axis_label = '% Patients having Diabetes'

p1.y_range = Range1d(0,65)
p1.x_range = Range1d(0,30)

callback_PB = CustomJS(args=dict(pb_source=pb_source, sources_diabetes=sources_diabetes), code="""
    var pb = pb_source.data
    var sources_diabetes = sources_diabetes.data
    var threshold = cb_obj.value
    y = [0,0]
    var lessCount = 0
    var moreCount = 0
    for(i=0; i < 54; i++){
        if ((sources_diabetes.diabetes[i] == 1) && (sources_diabetes.pb[i] < threshold)){
            y[0]+=1
            lessCount+=1
        }
        if ((sources_diabetes.diabetes[i] == 0) && (sources_diabetes.pb[i] < threshold)){
            lessCount+=1
        }
        if ((sources_diabetes.diabetes[i] == 1) && (sources_diabetes.pb[i] > threshold)){
            y[1]+=1
            moreCount+=1
        }
        if ((sources_diabetes.diabetes[i] == 0) && (sources_diabetes.pb[i] > threshold)){
            moreCount+=1
        }
    }
    y = [100.0*y[0]/lessCount, 100.0*y[1]/moreCount]
    pb.x[0] = threshold - 1
    pb.x[1] = threshold + 1
    pb.top = y
    pb_source.trigger('change');
""")
slider = Slider(start=min(df['PEANUTBUTTERVOL']), end=max(df['PEANUTBUTTERVOL']), value=15, step=1, title="Peanut Butter Volume Threshold", callback=callback_PB)

show(row(column(p1,slider),p2))

/home/sanuj/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/home/sanuj/anaconda3/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


### Diabetes
1. Oysters and Peanut Butter consumption were the factors found to affect diabetes occurence, with PB having a 0.8 weightage. To analyse this, the charts above were made.
2. The peanut butter chart shows on most threshold points that having a higher volume of peanut butter makes it less likely to have diabetes. The bar on the right of the threshold has a lower percentage of people having diabetes, and this is observed from T = 5 to T = 28.
3. This could be reasoned with the fact that peanut butter is generally healthy, and if substituted for other sugary snacks, it could lead to less sugar consumption.
4. Having more oysters could lead to less risk of diabetes.

In [145]:
outcome_var = 'heart_disease'
model = RandomForestClassifier(n_estimators=15, min_samples_split=15, max_depth=5, max_features=1)
predictor_var = list(df.columns.values)
# predictor_var = ['dog', 'favCable', 'cat', 'quit_smoking', 'readingMath'] #0-25
# predictor_var = ['EATMEAT']
max_CV = 0
max_predictors = []
for i in range(100):
    random_predictor = []
    for j in range(3):
        random_predictor.append(predictor_var[randint(3,26)])
    model = RandomForestClassifier(n_estimators=15, min_samples_split=15, max_depth=5, max_features=1)
    cv = prediction_model(model, df, random_predictor, outcome_var)
    if cv > max_CV:
        max_CV = cv
        max_predictors = random_predictor
        
print (max_CV, max_predictors)

0.74 ['dog', 'currently_smoke', 'favCable']


In [157]:
model = RandomForestClassifier(n_estimators=15, min_samples_split=15, max_depth=5, max_features=1)
predictors = ['dog', 'currently_smoke', 'favCable']
cv = prediction_model(model, df, predictors, outcome_var)
featimp = pd.Series(model.feature_importances_, index=predictors).sort_values(ascending=False)
print(cv)
print(featimp)
predictions = model.predict(df[predictors])

0.758181818182
dog                0.761052
currently_smoke    0.146853
favCable           0.092095
dtype: float64


### Heart Disease
The three factors above give us an accuracy of 75.8% in the prediction of 5-fold validated dataset.

In [192]:
hd_column = df['heart_disease']
hd_colors = []
for i in range(len(hd_column)):
    if hd_column[i] == 1:
        hd_colors.append('red')
    else:
        hd_colors.append('green')

y= [0,0,0,0,0,0]
for i in range(54):
    if df['dog'][i] == 0 and df['heart_disease'][i] == 1:
            y[0]+=1
    if df['dog'][i] == 0 and df['heart_disease'][i] == 0:
            y[1]+=1
    if df['currently_smoke'][i] == 0 and df['heart_disease'][i] == 1:
            y[2]+=1
    if df['currently_smoke'][i] == 0 and df['heart_disease'][i] == 0:
            y[3]+=1
    if df['favCable'][i] == 0 and df['heart_disease'][i] == 1:
            y[4]+=1
    if df['favCable'][i] == 0 and df['heart_disease'][i] == 0:
            y[5]+=1

data = {'data':
        [{
            'name': 'No to Dog', 'Yes': y[0], 'No': y[1]
        },{
            'name': 'No to Currently Smoke', 'Yes': y[2], 'No': y[3]
        },{
            'name': 'No to favCable', 'Yes': y[4], 'No': y[5]
        }
        ]
       }
data = df_from_json(data)
bar = Bar(data,
          values=blend('Yes', 'No', name='values', labels_name='value'),
          label=cat(columns='name', sort=False),
          stack=cat(columns='value', sort=False),
          color=color(columns='value', palette=['red', 'green'],
                      sort=False),
          legend='top_right',
          title="Factors affecting Heart Disease, Stacked by value",
          tooltips=[('Heart Disease', '@value'), ('Response', '@name')])


factors = ['Dog', 'Currently Smoke', 'favCable']
x =  [76, 15, 9]

dot = figure(title="%age Contribution of Variables to predicting Heart Disease (Accuracy 76%)", tools="", toolbar_location=None,
            y_range=factors, x_range=[0,100], height=200, width=400)

dot.segment(0, factors, x, factors, line_width=2, line_color="green", )
dot.circle(x, factors, size=15, fill_color="orange", line_color="green", line_width=3, )

show(column(dot, bar))

### Heart Disease - Visualizations
1. Three factors were found to affect heart disease. Out of dog, currently_smoke, favCable, dog seems to affect the decision the most. We explore this relation in the stacked bar chart.
2. Each Factor's negative responses are plotted here with the red bar representing patients having a heart disease, and the green one not having one. We can clearly see that out of those who do not have a dog, there are more without a heart disease than with. The same goes for those who do not smoke.
3. Smoking, therefore, can lead to heart disease, and so can owning a dog, as the graph shows you are more likely to not have a heart disease if you don't own a dog and don't smoke.
4. The difference between the two classes is more pronouned in the case of 'dog', however, and this explains why the decision tree gave the most weight to it.